In [2]:
import pandas as pd
import os
from datetime import datetime, timedelta

In [3]:
dir_path = 'D:/Training_BSE/marketing_analytics'
file_name = 'OnlineRetail.csv'

In [4]:
#Read the data
df_data = pd.read_csv(os.path.join(dir_path,file_name),encoding='iso-8859-1')

In [5]:
df_data.shape

(541909, 8)

In [8]:
df_data['InvoiceDate'] = pd.to_datetime(df_data['InvoiceDate'],errors='coerce')

In [10]:
df_data['CustomerID'] = df_data['CustomerID'].astype(str)

In [14]:
tx_uk = df_data.loc[df_data['Country']=='United Kingdom']

In [15]:
tx_uk.shape

(495478, 8)

In [17]:
tx_uk['CustomerID'] = tx_uk['CustomerID'].astype(str).str.split('.').str[0]

<ipython-input-17-7134117e3bc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tx_uk['CustomerID'] = tx_uk['CustomerID'].astype(str).str.split('.').str[0]


In [21]:
#Filter the Data for 6M and 3M from the last date in the data
max_date = tx_uk['InvoiceDate'].max()
tx_6M = tx_uk.loc[(tx_uk['InvoiceDate']<=max_date)&(tx_uk['InvoiceDate'] > (max_date-timedelta(days=180)))]

In [22]:
tx_6M.shape

(289998, 8)

In [24]:
tx_6M['Revenue'] = tx_6M['UnitPrice'] * tx_6M['Quantity']

<ipython-input-24-dc134382dd78>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tx_6M['Revenue'] = tx_6M['UnitPrice'] * tx_6M['Quantity']


In [25]:
tx_RFM = tx_6M.groupby(['CustomerID']).agg(MaxPurchaseDate=('InvoiceDate','max'),\
                                           Frequency=('InvoiceNo','count'),Monetary_Val = ('Revenue','sum')).reset_index()

In [26]:
tx_RFM.shape

(3172, 4)

In [27]:
tx_RFM['Recency'] = (tx_RFM['MaxPurchaseDate'].max() - tx_RFM['MaxPurchaseDate']).dt.days

In [39]:
#Sorting the data
tx_RFM = tx_RFM.sort_values(by=['Recency','Frequency','Monetary_Val'],ascending = [True,False,False])
tx_RFM = tx_RFM.loc[tx_RFM['Monetary_Val'] <= 70000]

In [43]:
from sklearn.cluster import KMeans
#Cluster the RFM part
kmeans = KMeans(n_clusters=3)
k_fit = kmeans.fit_predict(tx_RFM[['Recency','Frequency','Monetary_Val']])
tx_RFM['Cluster'] = k_fit

In [44]:
tx_RFM_grp_cluster = tx_RFM.groupby(['Cluster'])['Monetary_Val'].describe()

In [45]:
tx_RFM_grp_cluster

,count,mean,std,min,25%,50%,75%,max
Cluster,,,,,,,,
0,2947.0,700.829380,643.695846,-4287.63,239.6100,479.450,986.4300,2989.87
1,20.0,30228.545000,10741.205372,18052.00,22869.9850,27151.400,33738.0325,57120.91
2,202.0,5292.599802,2762.861925,3000.21,3534.6775,4083.825,5894.8600,16068.45


In [47]:
tx_RFM.to_excel(os.path.join(dir_path,'clustered.xlsx'),index=None)